In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
import astropy.constants as const
import astropy.units as u
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.modeling.models import Gaussian2D as AstropyGauss2d
from astropy.time import Time


from fastimgproto.sourcefind.fit import Gaussian2dParams
from fastimgproto.sourcefind.image import (
    SourceFindImage, IslandParams, PixelIndex, Pixel,)
#from fastimgproto.bindings.sourcefind import cpp_sourcefind

from fastimgproto.fixtures.image import (
    add_gaussian2d_to_image,
    gaussian_point_source,
    uncorrelated_gaussian_noise_background
)
from fastimgproto.fixtures.sourcefind import random_sources_on_grid
import math

In [ ]:
from tkp.sourcefinder.image import ImageData as TkpSFImage

In [ ]:
image_size = 64
image_shape = (image_size, image_size)
# ydim = 64
# xdim = 32
# image_shape = (ydim, xdim)

In [ ]:
n_sources=1
amplitude_range = (28., 42.)
semiminor_range = (1.2, 2.5)
axis_ratio_range = (1., 2.)
seed = 123456

In [ ]:
sources_grid = random_sources_on_grid(image_size,
                                     n_sources,
                                     amplitude_range=amplitude_range,
                                     semiminor_range=semiminor_range,
                                     axis_ratio_range=axis_ratio_range,
                                     seed=seed
                                )

In [ ]:
tough_source = Gaussian2dParams(
    x_centre=18.48820770355941, y_centre=34.05960466345559, amplitude=12.609448034550804,
    semimajor=2.913573230793465, semiminor=0.7230940150989847, theta=-1.4507887213254516)

test_source = gaussian_point_source(18, 34.5, amplitude=18.3, semimajor=2.5, semiminor=2.0, theta=0.25)
# img = uncorrelated_gaussian_noise_background(image_shape, sigma=0.1)
img = np.zeros(image_shape, dtype=np.float_)
noisemap = np.ma.MaskedArray(np.ones_like(img))
add_gaussian2d_to_image(test_source, img)


# for s in sources_grid:
#     add_gaussian2d_to_image(s, img)

In [ ]:
from tkp.testutil.mock import make_wcs
tkpimg = TkpSFImage(data=img, beam=(1,1,0), wcs=make_wcs(), residuals=False)
tkpimg._set_rm(noisemap)
tkpimg._set_backmap(np.zeros_like(img))

In [ ]:
results = tkpimg.extract(det=5,anl=3, noisemap=noisemap)

In [ ]:
results

In [ ]:
found=results[0]

In [ ]:
test_source

In [ ]:
hwhm_factor = math.sqrt(2*math.log(2))
hwhm_factor

In [ ]:
found.y, found.x, found.peak, found.smaj/hwhm_factor, found.smin/hwhm_factor, -1*found.theta